In [1]:
from flask import Flask, render_template, request
import serial
import threading

app = Flask(__name__,
            static_url_path='', 
            static_folder='static',
            template_folder='templates')
@app.route('/')

@app.get('/shutdown')
def shutdown():
    shutdown_server()
    return 'Server shutting down...'

def index():
    return render_template('index.html')

    
SERIAL_PORT = 'COM5'
SERIAL_RATE = 115200

class IMU:
    def _init_(self):
        self.AccXcal = 0
        self.AccYcal = 0
        self.AccZcal = 0

def readSerialData20():
    ser = []
    try:
        print("Opening port:", SERIAL_PORT)
        ser = serial.Serial(SERIAL_PORT, SERIAL_RATE)

    except serial.SerialException:
        print("Culd not open serial port!")
        exit
        
    while True:
        try:
            data = ser.read(20)
            #print(data)
            if data is not None:
                IMU.head = int.from_bytes(data[0:2],"little")
                IMU.AccX = int.from_bytes(data[2:4],"little", signed=True) #+-4g obcutljivost 2/(2^16/4)
                IMU.AccY = int.from_bytes(data[4:6],"little", signed=True) #stevila so v enotah 1G
                IMU.AccZ = int.from_bytes(data[6:8],"little", signed=True)
                IMU.GyroX = int.from_bytes(data[8:10],"little", signed=True)
                IMU.GyroY = int.from_bytes(data[10:12],"little", signed=True)
                IMU.GyroZ = int.from_bytes(data[12:14],"little", signed=True)
                IMU.MagX = int.from_bytes(data[14:16],"little", signed=True)
                IMU.MagY = int.from_bytes(data[16:18],"little", signed=True)
                IMU.MagZ = int.from_bytes(data[18:20],"little", signed=True)
                print(IMU.AccX,IMU.AccY,IMU.AccZ)
            
        except serial.SerialException:
            print("Closing serial")
            ser.close()
            break

if __name__ == '__main__':
    
    t1 = threading.Thread(target=readSerialData20, args=())
    t1.start()
    
    app.run(debug=True, port=6555, host='0.0.0.0')
    #t1.join()
    print("Done")

    
    

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with stat


SystemExit: 1

D:\WindowsPrograms\Conda\envs\Basic\lib\site-packages\IPython\core\interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
ser.close()

In [ ]:
def readSerialData20(ser):
    while True:
        sync = 0
        syncCount = 0
        while sync < 3:
            #byte_in = ser.read(20)
            
            byte_in = ser.read()
            print(byte_in)
            if sync == 0:
                if byte_in == b'\xAB':
                    sync = 1
                    syncCount = 0
                    imu_data_raw[0] = int.from_bytes(byte_in, byteorder='big')
                    continue
                    
            if sync == 1:
                if byte_in == b'\xAA':
                    sync = 2
                    syncCount = 1
                    imu_data_raw[1] = int.from_bytes(byte_in, byteorder='big')
                    continue
                    

            if sync ==2:
                syncCount += 1
                imu_data_raw[syncCount] = int.from_bytes(byte_in, byteorder='big')
                if syncCount >= PACKET_LENGHT-1:
                    number += 1
                    print(imu_data_raw,number)
                    break

            if syncCount == PACKET_LENGHT - 1:
                #print(imu_data_raw)
                sync = 3
